In [1]:
import string, random
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
import gzip
import pickle
import itertools
import queue as qe
import copy

from sklearn.linear_model import LinearRegression
from random import shuffle

import networkx as nx

In [2]:
mg = nx.MultiDiGraph()

mg.add_weighted_edges_from([(0, 1, 10), (0, 2, 1), (1, 2, 1), (2, 3, 1), (3, 1, 1)])
mg.add_weighted_edges_from([(0, 0, 10), (0, 2, 1), (1, 2, 1), (2, 3, 1), (3, 1, 1)])

print(mg[0])

{1: {0: {'weight': 10}}, 2: {0: {'weight': 1}, 1: {'weight': 1}}, 0: {0: {'weight': 10}}}


In [3]:
def m_mg_2_d_mg(m_mg):
    ret = {}
    
    for i in range(m_mg.shape[0]):
        ret.update({i:{}})
        for j in m_mg[i]:
            if j > 0:
                ret[i].update({j:{}})
                for k in range(j):
                    ret[i][j].update({k:1})
                    
    return ret
            
def rand_unweighted_multigraph(n_nodes, num_max_multiple_edges = 3, prob = 0.5):
    
    ramas = np.random.binomial(num_max_multiple_edges-1, 1-prob, size=(n_nodes, n_nodes)) 

    return m_mg_2_d_mg(ramas)

In [4]:
g = rand_unweighted_multigraph(4)

In [5]:
def graph_2_multigraph(g):
    ret = {}
    
    for u in g.keys():
        ret.update({u:{}})
        for v,i in g[u].items():
            ret[u].update({v:{0:i}})
    
    return ret

def print_multi_graph(g):
    for u in g.keys():
        for v in g[u].keys():
            l = []
            for i in g[u][v].keys():
                l.append(str(i)+':'+str(g[u][v][i]))
            print((u,v),"".join(str(x)+" " for x in l))

In [6]:
'''d_g = {
0: {1: 10, 2: 1}, 
1: {2: 1, 3: 5}, 
2: {3: 1},
3: {1: 1, 0:20}
}'''

#print(graph_2_multigraph(d_g)
#mg = graph_2_multigraph(d_g)
print(g)
print('\n\n')

print_multi_graph(g)

{0: {2: {0: 1, 1: 1}}, 1: {1: {0: 1}, 2: {0: 1, 1: 1}}, 2: {1: {0: 1}, 2: {0: 1, 1: 1}}, 3: {2: {0: 1, 1: 1}}}



(0, 2) 0:1 1:1 
(1, 1) 0:1 
(1, 2) 0:1 1:1 
(2, 1) 0:1 
(2, 2) 0:1 1:1 
(3, 2) 0:1 1:1 


In [7]:
def adj_inc_directed_multigraph(d_mg):
    
    adj = [0]*len(d_mg)
    inc = [0]*len(d_mg)
    
    for key_n,value_n in d_mg.items():
        for key_adj,value_adj in value_n.items():
            for keys_edg in value_adj.keys():
                adj[key_n]+=1
                inc[key_adj]+=1
    
    return adj,inc

def __get_u_v__(inc,adj):
    u = None
    v = None # índice del último nodo
     
    for i in range(len(adj)):
        # check for u
        if inc[i] == adj[i] - 1 and u == None:
            u = i
            
        # check for v
        if inc[i] == adj[i] + 1 and v == None:
            v = i
    
    return u,v

def isthere_euler_path_directed_multigraph(d_mg):
    """
    Seguimos el corolario de las notesGraphs_2018
    
    """
    adj, inc = adj_inc_directed_multigraph(d_mg)
   
    u,v = __get_u_v__(inc,adj)
    
    if u == None or v == None: # No tenemos nodo inicial/final
        return False
    
    return all([True if adj[n] == inc[n] or n == u or n == v else False for n in range(len(adj))])
    
    
def first_last_euler_path_directed_multigraph(d_mg):
    if not isthere_euler_path_directed_multigraph(d_mg): 
        return ()
    else:
        adj, inc = adj_inc_directed_multigraph(d_mg)
        u,v = __get_u_v__(inc,adj)
        
        return (u,v)

    
d_g_camino = { # sabemos que este grafo tiene camino euleriano
0: {1:{0:1}, 3:{0:1}}, 
1: {1:{0:1,1:1}, 2:{0:1}}, 
2: {0:{0:1}},
3: {}
}
    
adj, inc = adj_inc_directed_multigraph(g);
print(adj,"\n", inc)
print(isthere_euler_path_directed_multigraph(g))
adj, inc = adj_inc_directed_multigraph(d_g_camino);
print(adj,"\n", inc)
print(isthere_euler_path_directed_multigraph(d_g_camino))
print(first_last_euler_path_directed_multigraph(g))
print(first_last_euler_path_directed_multigraph(d_g_camino))
    

[2, 3, 3, 2] 
 [0, 2, 8, 0]
False
[2, 3, 1, 0] 
 [1, 3, 1, 1]
True
()
(0, 3)


In [233]:
d_g_camino = { # sabemos que este grafo tiene camino euleriano
0: {1:{0:1}, 3:{0:1}}, 
1: {1:{0:1,1:1}, 2:{0:1}}, 
2: {0:{0:1}},
3: {}
}

d_g_camino = { # sabemos que este grafo tiene camino euleriano
1: {2:{0:1}}, 
2: {3:{0:1}, 4:{0:1}}, 
3: {4:{0:1}},
4: {6:{0:1}, 7:{0:1}},
5: {2:{0:1}},
6: {},
7: {5:{0:1}}
}

def euler_walk_directed_multigraph(u, d_mg):
    if not d_mg or u == None:
        return []
    
    c = copy.deepcopy(d_mg)
    ret =  []
    
    while d_mg[u].keys():
        if not list(c[u].keys()):
            break
        
        z = list(c[u].keys())[0]
        
        p = list(c[u][z])[0]
        
        ret.append((u,z))
        
        c[u][z].pop(p)
        
        if not c[u][z].keys(): 
            c[u].pop(z)
        
        u = z
    
    return ret


def next_first_node(l_path,d_mg):
    if not l_path or not d_mg:
        return None
    
#     d_mg_copy = copy.deepcopy(d_mg)
    print("Camino: ", l_path)
    for tup in l_path:
        print('\nNext tup: ', tup)
#         if not d_mg[tup[0]]:
        if d_mg_copy[tup[0]]:  
            if len(d_mg_copy[tup[0]][tup[1]]) == 1:
                d_mg_copy[tup[0]].pop(tup[1])
                print('Removing-',tup)
            else:
                extract_key = list(d_mg_copy[tup[0]][tup[1]].keys())[0]
                # Extraemos la key
                d_mg_copy[tup[0]][tup[1]].pop(extract_key)
    
    print('\nEl dict queda: ',d_mg_copy)
    
    for k, v in d_mg_copy.items():
        if v.keys():
            print('Clave-valor',k,'-',v)
            return k
    
    return None


def path_stitch(path_1, path_2):
    if not path_1:
        return path_2
    if not path_2:
        return path_1
    if (not path_1) and (not path_2):
        return []
    
    ret = []
    
    for tup1 in path_1:
        ret.append(tup1)
        if tup1[1] == path_2[0][0]:
            for tup2 in path_2:
                ret.append(tup2)
    
    return ret

print(d_g_camino)
#print(euler_walk_directed_multigraph(0,d_g_camino))
print(euler_walk_directed_multigraph(1,d_g_camino))
#print(euler_walk_directed_multigraph(3,d_g_camino))
#print(next_first_node(euler_walk_directed_multigraph(0,d_g_camino), d_g_camino))
l1 = euler_walk_directed_multigraph(1,d_g_camino)
n = next_first_node(l1, d_g_camino)

# print(euler_walk_directed_multigraph(n,d_g_camino))
l2 = euler_walk_directed_multigraph(n,d_g_camino)
print(l2)
print(path_stitch(l1, l2))

{1: {2: {0: 1}}, 2: {3: {0: 1}, 4: {0: 1}}, 3: {4: {0: 1}}, 4: {6: {0: 1}, 7: {0: 1}}, 5: {2: {0: 1}}, 6: {}, 7: {5: {0: 1}}}
[(1, 2), (2, 3), (3, 4), (4, 6)]
Camino:  [(1, 2), (2, 3), (3, 4), (4, 6)]

Next tup:  (1, 2)
Removing- (1, 2)

Next tup:  (2, 3)
Removing- (2, 3)

Next tup:  (3, 4)
Removing- (3, 4)

Next tup:  (4, 6)
Removing- (4, 6)

El dict queda:  {1: {}, 2: {4: {0: 1}}, 3: {}, 4: {7: {0: 1}}, 5: {2: {0: 1}}, 6: {}, 7: {5: {0: 1}}}
Clave-valor 2 - {4: {0: 1}}
[(2, 3), (3, 4), (4, 6)]
[(1, 2), (2, 3), (3, 4), (4, 6), (2, 3), (3, 4), (4, 6)]


In [205]:
d_g_camino = { # sabemos que este grafo tiene camino euleriano
0: {1:{0:1}},
1: {2:{0:1}}, 
2: {3:{0:1}, 4:{0:1}}, 
3: {4:{0:1}},
4: {6:{0:1}, 7:{0:1}},
5: {2:{0:1}},
6: {},
7: {5:{0:1}}
}

def euler_path_directed_multigraph(d_mg):
    # Comprobamos que tenga camino euleriano
    if not isthere_euler_path_directed_multigraph(d_mg):
        return []
    
    # Sacamos el nodo inicial y el nodo final 
    (first,last) = first_last_euler_path_directed_multigraph(d_mg)
    
    l1 = euler_walk_directed_multigraph(first, d_mg)
    n = next_first_node(l1,d_mg)
    
    while n:
        l = euler_walk_directed_multigraph(n,d_mg)
        
        l1 = path_stitch(l1,l)
        
        n = next_first_node(l1,d_mg)
        print('\nSiguiente nodo: ',n)
    
    return l1

# def __check_euler_path(final_path,d_mg):
#     ramas = len(final_path)
#     ramas_grafo = 0
    
#     for key,value in d_mg.items():
#         ramas_grafo += len(value.keys())
    
#     print (ramas,ramas_grafo)

l1 = euler_path_directed_multigraph(d_g_camino)
# __check_euler_path(l1,d_g_camino)
print(l1)

Camino:  [(0, 1), (1, 2), (2, 3), (3, 4), (4, 6)]

Next tup:  (0, 1)

Next tup:  (1, 2)

Next tup:  (2, 3)

Next tup:  (3, 4)

Next tup:  (4, 6)

El dict queda:  {0: {}, 1: {}, 2: {4: {0: 1}}, 3: {}, 4: {7: {0: 1}}, 5: {2: {0: 1}}, 6: {}, 7: {5: {0: 1}}}
Camino:  [(0, 1), (1, 2), (2, 4), (4, 7), (7, 5), (5, 2), (2, 3), (3, 4), (4, 6)]

Next tup:  (0, 1)

Next tup:  (1, 2)

Next tup:  (2, 4)

Next tup:  (4, 7)

Next tup:  (7, 5)

Next tup:  (5, 2)

Next tup:  (2, 3)

Next tup:  (3, 4)

Next tup:  (4, 6)

El dict queda:  {0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}}

Siguiente nodo:  None
[(0, 1), (1, 2), (2, 4), (4, 7), (7, 5), (5, 2), (2, 3), (3, 4), (4, 6)]


In [206]:
d_g_camino = { # sabemos que este grafo tiene camino euleriano
0: {1:{0:1}},
1: {2:{0:1}}, 
2: {3:{0:1}}, 
3: {4:{0:1}},
4: {5:{0:1}},
5: {6:{0:1}},
6: {7:{0:1}},
7: {0:{0:1}}
}

def isthere_euler_circuit_directed_multigraph(d_mg):
    adj,inc = adj_inc_directed_multigraph(d_mg)
    
    if not adj == inc:
        return False
    else:
        return True

isthere_euler_circuit_directed_multigraph(d_g_camino)

def euler_circuit_directed_multigraph(d_mg, u=0):
    if not isthere_euler_circuit_directed_multigraph(d_mg):
        return []
    
    l1 = euler_walk_directed_multigraph(u, d_mg)
    n = next_first_node(l1,d_mg)
    
    while n:
        l = euler_walk_directed_multigraph(n,d_mg)
        
        l1 = path_stitch(l1,l)
        
        n = next_first_node(l1,d_mg)
    
    return l1

print(euler_circuit_directed_multigraph(d_g_camino))

Camino:  [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 0)]

Next tup:  (0, 1)

Next tup:  (1, 2)

Next tup:  (2, 3)

Next tup:  (3, 4)

Next tup:  (4, 5)

Next tup:  (5, 6)

Next tup:  (6, 7)

Next tup:  (7, 0)

El dict queda:  {0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}}
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 0)]


In [12]:
#Cuestion

In [207]:
def random_sequence(len_seq):
    seq = ['A', 'C', 'G', 'T']
    
    np.random.seed()
    
    return ''.join([seq[np.random.randint(0,4)] for n in range(len_seq)])

s = random_sequence(14)
print('Cadena aleatoria: ' + s)

def spectrum(sequence, len_read):
        
    ret = [sequence[i:i+len_read] for i in range(len(sequence)) if ((i+len_read) <= len(sequence))]
    
    shuffle(ret)
    
    return ret

s = spectrum(s, 3)
# print('\nEspectro X: ', s)

def spectrum_2(spectr):
    sp_list = [spectr[i][j:j+len(spectr[0])-1] for i in  range(len(spectr)) for j in [0,1]]
    sp_menos_1 = []
    
    for i in sp_list:
        if i not in sp_menos_1:
            sp_menos_1.append(i) # Quitamos los duplicados porque un set lo desordena
    
    return sp_menos_1

# print('\nEspectro X-1: ',set(spectrum_2(s)))

def __can_combine(s1,s2):
    if len(s1) == 1: 
        return True
    
    take = len(s1)-1
#     print('can combine',s1,s2,'?')
#     print(s1[-take:],s2[:take])
#     print(s1[-take:] == s2[:take])
    
    return s1[-take:] == s2[:take]

# print(__can_combine('TT','CC'))
# print(__can_combine('AC','GT'))
# print(__can_combine('AG','GC'))

def __combine_spectr(s1,s2):
    if len(s1) == 1:
        return s1+s2
    
    take = len(s1)-1
#     print('lets combine ',s1,s2)
#     print('combine:',s1[:take]+s2)
    return s1[:take]+s2

def spectrum_2_undirected_graph(spectr):
    print('\nEspectro X: ', spectr)
    sp_list = spectrum_2(spectr)
                
    print('\nEspectro X-1: ',sp_list)
    
#     ramas_x_vuelta = 0
    nodos = range(len(sp_list))
    
    d1 = {n:{} for n in nodos}
      
    for s1 in list(d1.keys()):
        n1 = sp_list[s1]
#         print('n1 = ', n1,'s1 = ',s1)
        for s2 in list(d1.keys()):
            n2 = sp_list[s2]
#             print('n2 = ',n2,'s2 = ',s2)
#             print('from: ',n1,' to ',n2)
            if (__can_combine(n1,n2)) and (__combine_spectr(n1,n2) in spectr):
#                 print(__combine_spectr(n1,n2),' is in spectre')
                edges_count = spectr.count(__combine_spectr(n1,n2)) #Obtenemos el número de ramas que van de un nodo a otro
                cont = 0
                d1[s1].update({s2:{}})
                while cont < edges_count:
                    
                    d1[s1][s2].update({cont:1}) #Actualizamos el diccionario de nodo destino con el número de ramas correspondiente
                    cont += 1
#                 print('Estado del dict: ',d1)
#                     ramas_x_vuelta+=1
        
#             ramas_x_vuelta = 0
        
    return d1
    
dir_graph = spectrum_2_undirected_graph(s)

print('\nMultigrafo dirigido: ',dir_graph)

Cadena aleatoria: ATAAGTGGAGTGCG

Espectro X:  ['GGA', 'AGT', 'TGC', 'TAA', 'GAG', 'ATA', 'AAG', 'GTG', 'GTG', 'TGG', 'GCG', 'AGT']

Espectro X-1:  ['GG', 'GA', 'AG', 'GT', 'TG', 'GC', 'TA', 'AA', 'AT', 'CG']

Multigrafo dirigido:  {0: {1: {0: 1}}, 1: {2: {0: 1}}, 2: {3: {0: 1, 1: 1}}, 3: {4: {0: 1, 1: 1}}, 4: {0: {0: 1}, 5: {0: 1}}, 5: {9: {0: 1}}, 6: {7: {0: 1}}, 7: {2: {0: 1}}, 8: {6: {0: 1}}, 9: {}}


In [208]:
spectre = ['CAA','AAG']
spectre_2 = ['CA','AA','AG']

print(spectrum_2_undirected_graph(spectre))
print(spectrum_2_undirected_graph(spectre_2))


Espectro X:  ['CAA', 'AAG']

Espectro X-1:  ['CA', 'AA', 'AG']
{0: {1: {0: 1}}, 1: {2: {0: 1}}, 2: {}}

Espectro X:  ['CA', 'AA', 'AG']

Espectro X-1:  ['C', 'A', 'G']
{0: {1: {0: 1}}, 1: {1: {0: 1}, 2: {0: 1}}, 2: {}}


In [209]:
def path_2_sequence(l_path, spectrum_2):    
    seq = spectrum_2[l_path[0][0]]
    
    for tup in l_path:
        seq = seq+spectrum_2[tup[1]][-1]
    
    return seq


def check_sequencing(len_seq, len_read):
    
    assert len_read > 1, "No se puede pasar 1 o menos como argumento de 'len_read'"
    
    s = random_sequence(len_seq) #Se genera una secuencia aleatoria de tamaño len_seq
    print("Cadena generada: ",s)
    
    spec = spectrum(s, len_read)
    
    spec_2 = spectrum_2(spec)
    
    print("l-espectro generado: ",spec)
    
    print("(l-1)-espectro: ",spec_2)

    d_mg = spectrum_2_undirected_graph(spec)
    
    print("Multigrafo generado: ",d_mg)
    
    camino = isthere_euler_path_directed_multigraph(d_mg)
    circuito = isthere_euler_circuit_directed_multigraph(d_mg)
    
    if (not camino) and (not circuito):
        print("No se puede reconstruir la secuencia dado que no hay camino euleriano")
        return False #Si no hay camino euleriano, la cadena no se podrá reconstruir correctamente
    elif camino:
        path = euler_path_directed_multigraph(d_mg)
        
        print("Camino euleriano: ",path)
        
        seq_2 = path_2_sequence(path, spec_2)
        
        print(seq_2)
        
        return seq_2 == s
    else:
        print('circuito hay')
    
_=check_sequencing(20, 3)
        

Cadena generada:  CAGCAGGTTTCGGCTTATAG
l-espectro generado:  ['GGT', 'TTC', 'TCG', 'CGG', 'CAG', 'AGG', 'GCT', 'CTT', 'GCA', 'TAG', 'CAG', 'AGC', 'TTA', 'TTT', 'GTT', 'ATA', 'GGC', 'TAT']
(l-1)-espectro:  ['GG', 'GT', 'TT', 'TC', 'CG', 'CA', 'AG', 'GC', 'CT', 'TA', 'AT']

Espectro X:  ['GGT', 'TTC', 'TCG', 'CGG', 'CAG', 'AGG', 'GCT', 'CTT', 'GCA', 'TAG', 'CAG', 'AGC', 'TTA', 'TTT', 'GTT', 'ATA', 'GGC', 'TAT']

Espectro X-1:  ['GG', 'GT', 'TT', 'TC', 'CG', 'CA', 'AG', 'GC', 'CT', 'TA', 'AT']
Multigrafo generado:  {0: {1: {0: 1}, 7: {0: 1}}, 1: {2: {0: 1}}, 2: {2: {0: 1}, 3: {0: 1}, 9: {0: 1}}, 3: {4: {0: 1}}, 4: {0: {0: 1}}, 5: {6: {0: 1, 1: 1}}, 6: {0: {0: 1}, 7: {0: 1}}, 7: {5: {0: 1}, 8: {0: 1}}, 8: {2: {0: 1}}, 9: {6: {0: 1}, 10: {0: 1}}, 10: {9: {0: 1}}}
Camino:  [(5, 6), (6, 0), (0, 1), (1, 2), (2, 2), (2, 3), (3, 4), (4, 0), (0, 7), (7, 5), (5, 6), (6, 7), (7, 8), (8, 2), (2, 9), (9, 6)]

Next tup:  (5, 6)

Next tup:  (6, 0)

Next tup:  (0, 1)

Next tup:  (1, 2)

Next tup:  (2, 2

In [181]:
def check_sequencing_2(s, len_read):
    
    assert len_read > 1, "No se puede pasar 1 o menos como argumento de 'len_read'"
    
#     s = random_sequence(len_seq) #Se genera una secuencia aleatoria de tamaño len_seq
#     print("Cadena generada: ",s)
    
    spec = spectrum(s, len_read)
    
    spec_2 = spectrum_2(spec)
    
    print("l-espectro generado: ",spec)
    
    print("(l-1)-espectro: ",spec_2)

    d_mg = spectrum_2_undirected_graph(spec)
    
    print("Multigrafo generado: ",d_mg)
    
    camino = isthere_euler_path_directed_multigraph(d_mg)
    circuito = isthere_euler_circuit_directed_multigraph(d_mg)
    
    if (not camino) and (not circuito):
        print("No se puede reconstruir la secuencia dado que no hay camino euleriano")
        return False #Si no hay camino euleriano, la cadena no se podrá reconstruir correctamente
    elif camino:
        path = euler_path_directed_multigraph(d_mg)
        
        print("Camino euleriano: ",path)
        
        seq_2 = path_2_sequence(path, spec_2)
        
        print(seq_2)
        
        return seq_2 == s
    else:
        print('circuito hay')

In [234]:
a = {0:{}}

print(a[0].keys())

dict_keys([])


In [236]:
cadena = 'CAGCACCACCTGATTAGCCC'

random.seed()

print(check_sequencing_2(cadena,3))

l-espectro generado:  ['TTA', 'CCC', 'CTG', 'GAT', 'CAC', 'TGA', 'GCA', 'ACC', 'CAG', 'ATT', 'CAC', 'CCA', 'AGC', 'AGC', 'CCT', 'ACC', 'GCC', 'TAG']
(l-1)-espectro:  ['TT', 'TA', 'CC', 'CT', 'TG', 'GA', 'AT', 'CA', 'AC', 'GC', 'AG']

Espectro X:  ['TTA', 'CCC', 'CTG', 'GAT', 'CAC', 'TGA', 'GCA', 'ACC', 'CAG', 'ATT', 'CAC', 'CCA', 'AGC', 'AGC', 'CCT', 'ACC', 'GCC', 'TAG']

Espectro X-1:  ['TT', 'TA', 'CC', 'CT', 'TG', 'GA', 'AT', 'CA', 'AC', 'GC', 'AG']
Multigrafo generado:  {0: {1: {0: 1}}, 1: {10: {0: 1}}, 2: {2: {0: 1}, 3: {0: 1}, 7: {0: 1}}, 3: {4: {0: 1}}, 4: {5: {0: 1}}, 5: {6: {0: 1}}, 6: {0: {0: 1}}, 7: {8: {0: 1, 1: 1}, 10: {0: 1}}, 8: {2: {0: 1, 1: 1}}, 9: {2: {0: 1}, 7: {0: 1}}, 10: {9: {0: 1, 1: 1}}}
Camino:  [(7, 8), (8, 2), (2, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 0), (0, 1), (1, 10), (10, 9), (9, 2), (2, 7), (7, 8), (8, 2)]

Next tup:  (7, 8)

Next tup:  (8, 2)

Next tup:  (2, 2)
Removing- (2, 2)

Next tup:  (2, 3)
Removing- (2, 3)

Next tup:  (3, 4)
Removing- (3, 4)



KeyError: 2

In [151]:
random.seed(2)

print(random.randint(12,50),random.randint(12,50),random.randint(12,50),random.randint(12,50))

15 17 17 35
